# Trabalho Prático 1: Machine Learning Algorithms (2025-2026)
## Mestrado em Inteligência Artificial Aplicada (MAAI) - IPCA

**Estudante:** Luã Pieroni Mariano | 19725
**Data:** Janeiro, 2026
**Tema:** Previsão de Preços de Jogos da Steam (Regressão)

---
### Resumo
Este notebook documenta o processo de desenvolvimento de um modelo inteligente para prever preços de jogos na plataforma Steam.
O projeto evoluiu em duas grandes iterações:
1.  **Iteração Manual (Prototipagem):** Análise exploratória e feature engineering manual.
2.  **Iteração Robusta (Pipeline):** Automatização do tratamento de dados (NLP para tags/gêneros) e seleção de modelo.

O modelo final (Random Forest) foi exportado para *deployment* via API (FastAPI).

## 1. Definição do Problema

**Objetivo:** Desenvolver um modelo de regressão capaz de prever o preço de um jogo baseado em características como género, data de lançamento, *publisher*, e *tags*.

**Contexto:**
A precificação de jogos digitais é complexa. O objetivo é utilizar dados históricos para inferir valores de mercado, auxiliando *developers* independentes ou analistas de mercado.

**Tipo de Problema:** Aprendizagem Supervisionada (Regressão).
**Métrica de Sucesso:** RMSE (Root Mean Squared Error) e R² Score.

### 2. Imports Gerais

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Pipeline Imports
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer

### 2.1 Leitura e Análise Inicial

In [ ]:
def Read_Data():
    """
    Lê o dataset original do CSV.
    
    Returns:
        tuple: (df original, df cópia para limpeza)
    """

    df = pd.read_csv("a_steam_data_2021_2025.csv")
    df_clean = df.copy()
    
    print("DATASET ORIGINAL")
    print(f"Dimensoes: {df.shape}")
    print(f"Colunas: {df.columns.tolist()}")
    
    return df, df_clean

def Analyze_Data(df_clean):
    """
    Exibe estatísticas descritivas e tipos de dados do dataset.
    """

    numericas = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    categoricas = df_clean.select_dtypes(include=['object']).columns.tolist()

    variables = {
        'Numericas': numericas,     
        'Categoricas': categoricas, 
        'Texto/Listas': ['genres', 'tags', 'categories'] # Colunas que precisam de tratamento especial
    }
    print("\n--- CLASSIFICAÇÃO DAS VARIÁVEIS ---")
    print(f"Numéricas ({len(numericas)}): {numericas}")
    print(f"Categóricas ({len(categoricas)}): {categoricas}")


    print("\n--- ESTATÍSTICAS DESCRITIVAS (Antes da Limpeza) ---")
    print(df_clean.describe(include='all')) 
    
    print("\n--- VERIFICAÇÃO DE DADOS FALTANTES ---")
    print(df_clean.isnull().sum())

    total_duplicados = df_clean.duplicated().sum()
    print(f"Linhas duplicadas: {total_duplicados}")

    return df_clean

### 2.2 Limpeza e Tratamento de Outliers

In [ ]:
def Clear_Data(df_clean):
    """
    Realiza a limpeza manual dos dados: remove duplicatas, nulos e trata outliers de preço.
    """

    #clear data
    df_clean = df_clean.drop_duplicates() #n temos duplicatas mas vou deixar com
    df_clean = df_clean.dropna(subset=['genres']) #remove os jogos sem generos
    df_clean = df_clean.dropna(subset=['developer', 'publisher','categories']) #remove os jogos sem dev e publisher, poderiamos mudar para Unknown
    df_clean = df_clean.dropna(subset=['price'])
    #tratando outliers pq o randon perdeu // nao foi necessario apenas em limpar a coluna appid ja foi o suficiente pra ganhar // é necessario para melhorar o r2
    df_clean = df_clean[df_clean['price'] < 100]

    if 'release_year' in df_clean.columns:
        df_clean['release_year'] = pd.to_numeric(df_clean['release_year'], errors='coerce')
        df_clean = df_clean.dropna(subset=['release_year']) # Remove os que deram erro

    #preenche numeros nos numericos restantes
    cols_numericas = df_clean.select_dtypes(include=[np.number]).columns
    df_clean[cols_numericas] = df_clean[cols_numericas].fillna(0)
    
    print(f"Total de linhas após limpeza: {df_clean.shape[0]}")

    print("\nDADOS NULOS APOS LIMPEZA")
    print(df_clean.isnull().sum())
    
    return df_clean